# Encoding & Decoding Python Objects with JSON

This guide will walk through creating custom encoders when dumping dictionaries to JSON objects. 

Let us create a dictionary containing nested python objects.  The JSON module can only convert basic python types by default.  In this notebook, I plan to demonstrate how to create a custom encoder to dump python dictionaries that contain object types. 

Below is an table of the default supported types:

| Python      | JSON        |
| ----------- | ----------- |
| dict      | ojbect        |
| list, tuple   | array        |
|str  | str        |
|int, float, int- & float-derived Enums  | number    |
|True  | true        |
|False  | false        |
|None  | null        |




Custom Python objects will have an issue serializing and will require custom encoders to process those types. 

First, let's set up the environment by importing the JSON module. In this example, our dictionary will contain an item with a custom class called "Decimal."  When invoking the JSON dumps method, we will illustrate what happens when you do not have a custom decoder configured.  



In [2]:
import json 

# Step 1
class Decimal:
    def __init__(self, num):
        self.num = num
        
    def __repr__(self):
        return f'< Decimal(num={self.num}) >'

    
# Step 2
my_decimal = Decimal(5)

data = {
    "item1": my_decimal,
    "item2": 5
}

try:
    json.dumps(data)
except TypeError as err:
  
    
    print(str(err))
    


Object of type Decimal is not JSON serializable


The message above describes the TypeError exception raising. The is due to the "item1 key" containing the value of an instance of a Decimal object. 

Next, let us create a serialization function that will be able to handle the encoding to JSON. This function will check if the object is of the correct type. The function will return the converted basic python type. 

Let's pass the newly created serialize function to the 'default' parameter to the JSON dumps function call.

In [57]:
def encode(obj):
    if isinstance(obj, Decimal):
        return int(obj.num)
    return obj.__dict__


def object_hook(obj):
    
    if isinstance(obj, dict):
       for key, val in obj.items():
            if isinstance(val, int):
                obj[key] = Decimal(val)
            
    return obj


    
# Encodes the 
json_str = json.dumps(data, default=encode)

json_data = json.loads(json_str, object_hook=object_hook)
print(json_str)
print(json_data)


{"item1": 5, "item2": 5}
{'item1': < Decimal(num=5) >, 'item2': < Decimal(num=5) >}


The above output displays the python type of the resulting JSON dumps calls as a string instead of a dictionary.  Binary data is transferred across the wire when using JSON. JSON represents its data as a human-readable string.  If you would like to access the properties of the JSON object, you will not be able to do so unless you reconvert the string back into a python object.

In [175]:
class Router:
    def __init__(self, **kwargs):
        self.name = kwargs.get('name')
        self.vendor = kwargs.get('vendor')
        
    def __repr__(self):
        return f'< Router(name={self.name}, vendor={self.vendor}) >'
  

class Switch:
    def __init__(self, **kwargs):
        self.name = kwargs.get('name')
        self.vendor = kwargs.get('vendor')
        
    def __repr__(self):
        return f'< Switch(name={self.name}, vendor={self.vendor}) >'

def generate_nodes(count=50):
    return [{'name': f'Node{i}', 'vendor': random.choice(vendors), '__class__': random.choice(classes)} for i in range(count)]

    
vendors = ['cisco', 'juniper', None]
classes = ['rtr', 'sw', 'bad_class']
import random


routers_objs = generate_nodes(50)


In [189]:
class MyDecoder(json.JSONDecoder):
    """
    Decodes the JSON string into a ptyhon object.  
    """
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        if isinstance(obj, dict):
            for key, val in obj.items():
                if isinstance(val, int):
                    obj[key] = Decimal(val)
        
        if '__class__' in obj.keys():
            if obj.get("__class__") == 'rtr' :
                return Router(name=obj.get("name"), vendor=obj.get("vendor"))
            
            if obj.get("__class__") == 'sw' :
                return Switch(name=obj.get("name"), vendor=obj.get("vendor"))
            
        return obj

    
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return int(obj.num)
        
        return obj.__dict__
    

    
def json_loads(data):
    if isinstance(data, dict):
        return MyDecoder().decode(json.dumps(data))
    
    if isinstance(data, bytes) or isinstance(data, str):
        return MyDecoder().decode(data)

def json_dumps(obj):
    return MyEncoder().encode(obj)


# Encodes the 
json_dump_results = json_dumps(data)
json_loads_results = [json_loads(router) for router in routers_objs]


def _filter_bad_classes(result):
    if isinstance(result, dict):
        if result.get("__class__") == 'bad_class':
            return True
        else:
            return False
        
    return False

def _filter_cisco_classes(result):

    if isinstance(result, Router) or isinstance(result, Switch):
        if result.vendor in vendors:
            return True
        else:
            return False
        
    return False

def filter_bad_classes(json_results):
    return list(filter(_filter_bad_classes, json_results))

def filter_cisco_classes(json_results):
    return list(filter(_filter_cisco_classes, json_results))

# print(json_dump_results)
print(filter_bad_classes(json_loads_results))

print(filter_cisco_classes(json_loads_results))

json_loads_results

[{'name': 'Node3', 'vendor': 'juniper', '__class__': 'bad_class'}, {'name': 'Node7', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node8', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node13', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node15', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node16', 'vendor': 'juniper', '__class__': 'bad_class'}, {'name': 'Node20', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node30', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node31', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node33', 'vendor': 'juniper', '__class__': 'bad_class'}, {'name': 'Node34', 'vendor': 'juniper', '__class__': 'bad_class'}, {'name': 'Node35', 'vendor': 'juniper', '__class__': 'bad_class'}, {'name': 'Node39', 'vendor': 'juniper', '__class__': 'bad_class'}, {'name': 'Node41', 'vendor': None, '__class__': 'bad_class'}, {'name': 'Node44', 'vendor': 'cisco', '__class__': 'bad_class'}, {'name': 'Node49', 'vendor': None, '__c

[< Router(name=Node0, vendor=None) >,
 < Switch(name=Node1, vendor=None) >,
 < Router(name=Node2, vendor=None) >,
 {'name': 'Node3', 'vendor': 'juniper', '__class__': 'bad_class'},
 < Router(name=Node4, vendor=cisco) >,
 < Switch(name=Node5, vendor=cisco) >,
 < Router(name=Node6, vendor=cisco) >,
 {'name': 'Node7', 'vendor': None, '__class__': 'bad_class'},
 {'name': 'Node8', 'vendor': None, '__class__': 'bad_class'},
 < Switch(name=Node9, vendor=juniper) >,
 < Router(name=Node10, vendor=cisco) >,
 < Router(name=Node11, vendor=juniper) >,
 < Router(name=Node12, vendor=cisco) >,
 {'name': 'Node13', 'vendor': None, '__class__': 'bad_class'},
 < Router(name=Node14, vendor=juniper) >,
 {'name': 'Node15', 'vendor': None, '__class__': 'bad_class'},
 {'name': 'Node16', 'vendor': 'juniper', '__class__': 'bad_class'},
 < Switch(name=Node17, vendor=cisco) >,
 < Router(name=Node18, vendor=None) >,
 < Switch(name=Node19, vendor=juniper) >,
 {'name': 'Node20', 'vendor': None, '__class__': 'bad_clas